In [27]:
from fastai.text.all import *
path = Path("data")

In [28]:
files = get_text_files(path, folders = ['train', 'test'])
txts = L(["Fourscore and seven years ago, our fathers brought forth upon this continent a new nation, conceived in liberty, and dedicated to the pursuit of happiness.",
          "No, you don't understand!"])
txt = txts[0]

In [29]:
tkn = Tokenizer(WordTokenizer())
toks = tkn(txt)
print(coll_repr(toks, 30))

(#31) ['xxbos','xxmaj','fourscore','and','seven','years','ago',',','our','fathers','brought','forth','upon','this','continent','a','new','nation',',','conceived','in','liberty',',','and','dedicated','to','the','pursuit','of','happiness'...]


In [30]:
toks_all = txts.map(tkn)
toks_all

(#2) [['xxbos', 'xxmaj', 'fourscore', 'and', 'seven', 'years', 'ago', ',', 'our', 'fathers', 'brought', 'forth', 'upon', 'this', 'continent', 'a', 'new', 'nation', ',', 'conceived', 'in', 'liberty', ',', 'and', 'dedicated', 'to', 'the', 'pursuit', 'of', 'happiness', '.'],['xxbos', 'xxmaj', 'no', ',', 'you', 'do', "n't", 'understand', '!']]

In [31]:
num = Numericalize(min_freq=1)
num.setup(toks_all)
print(coll_repr(num.vocab,20))

(#48) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj',',','and','fourscore','seven','years','ago','our','fathers','brought','forth','upon'...]


In [32]:
nums = num(toks)[:20]; nums

TensorText([ 2,  8, 11, 10, 12, 13, 14,  9, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         9, 25])

In [33]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj fourscore and seven years ago , our fathers brought forth upon this continent a new nation , conceived'

In [34]:
nums_all = toks_all.map(num)

In [35]:
dl = LMDataLoader(nums_all)

Creating the language model:

In [36]:
get_files = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_files, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [37]:
dls_lm.show_batch(max_n=2)

/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))


,text,text_
0,"xxbos xxunk .. xxunk .. xxmaj well you do n't have to xxunk if you can just tell me . xxbos a xxunk xxmaj x - ray diffraction pattern of as - prepared nafe0.5co0.5o2 was collected at xxunk line xxup xxunk , xxunk in xxmaj xxunk , xxunk with a large xxunk - xxunk xxunk [ 9 ] . xxbos xxmaj hey , xxmaj elaine , your friend never xxunk up xxmaj mr . xxmaj xxunk . xxbos xxmaj the","xxunk .. xxunk .. xxmaj well you do n't have to xxunk if you can just tell me . xxbos a xxunk xxmaj x - ray diffraction pattern of as - prepared nafe0.5co0.5o2 was collected at xxunk line xxup xxunk , xxunk in xxmaj xxunk , xxunk with a large xxunk - xxunk xxunk [ 9 ] . xxbos xxmaj hey , xxmaj elaine , your friend never xxunk up xxmaj mr . xxmaj xxunk . xxbos xxmaj the xxup"
1,"other important parameters for the transition from surface xxunk ( xxunk ) to early placode xxunk . xxbos xxmaj this research was supported in part by the xxmaj national xxmaj institute of xxmaj xxunk and xxmaj xxunk and xxmaj xxunk xxmaj diseases xxmaj beta xxmaj cell xxmaj xxunk xxmaj xxunk ( xxunk - xxunk ) and the xxmaj xxunk xxmaj institute for xxmaj xxunk xxmaj medicine ( xxunk - xxunk ) . xxbos xxmaj however , their energies should only","important parameters for the transition from surface xxunk ( xxunk ) to early placode xxunk . xxbos xxmaj this research was supported in part by the xxmaj national xxmaj institute of xxmaj xxunk and xxmaj xxunk and xxmaj xxunk xxmaj diseases xxmaj beta xxmaj cell xxmaj xxunk xxmaj xxunk ( xxunk - xxunk ) and the xxmaj xxunk xxmaj institute for xxmaj xxunk xxmaj medicine ( xxunk - xxunk ) . xxbos xxmaj however , their energies should only extend"


Fine-tuning the language model:

In [38]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [39]:
learn.fit_one_cycle(1, 2e-2)
learn.save_encoder('finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.495585,4.149165,0.275441,63.381065,00:02


In [40]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [41]:
dls_clas.show_batch(max_n=3)

/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/per202/src/fastai/fastai/torch_core.py:476: FutureWarning: The series.append method is deprecated 

,text,category
0,"xxbos xxmaj using xxmaj lemma 3 with xxunk = xxunk < xxunk / 2 = xxunk and xxunk = xxunk we see that the first lower bound is uniformly better ( larger ) than the second one , whereas again with xxmaj lemma 3 , neither of the upper bounds xxup gν+1 / 2,ν+1 / 2 and xxup gν , ν+2 is uniformly better ( smaller ) than the other : in fact , with xxunk - xxunk / xxunk - xxunk / 2 and xxunk / 2 , we get xxunk / 4 - 1 / xxunk + xxunk / 4 - 1 / xxunk / xxunk ) , so that gν , xxunk / 2,ν+1 / xxunk ) for 0 < s < t and xxup gν+1 / 2,ν+1 / xxunk , xxunk ) for s > t.",formal
1,"xxbos xxmaj the following primers were used : xxmaj xxunk / 4 : forward xxunk xxrep 5 g xxup xxunk xxrep 5 c xxup xxunk , reverse xxunk ; xxmaj sox17 : forward xxunk , reverse xxunk xxrep 3 a xxup xxunk ; xxunk - xxunk ( xxunk ): forward xxunk , reverse xxunk xxrep 3 a xxup xxunk ; xxunk ( xxunk ): forward xxunk xxrep 3 g xxup xxunk , reverse xxunk xxrep 3 t xxup xxunk xxrep 3 c xxup xxunk xxrep 4 t xxup xxunk ; xxup ttr : forward xxunk , reverse xxunk xxrep 3 c xxup xxunk xxup xxunk ; and xxup xxunk : forward xxunk xxrep 3 c xxup xxunk xxrep 4 a xxup xxunk , reverse xxunk",formal
2,"xxbos xxmaj the model solves the one - dimensional equations of motion for an escaping atmosphere composed of several neutral and ionized xxunk - xxunk xxunk is the density of species s , v is the vertical velocity , xxmaj fs is the xxunk flux of species s , xxmaj xxunk is the net chemical source term for species s , xxunk is a force term xxunk from viscous acceleration , e = xxunk is the specific internal energy of the gas , xxup xxunk is the specific net radiative heating rate , xxunk is the coefficient of heat xxunk , and xxunk is the viscous xxunk functional ( e.g. , xxunk and xxmaj xxunk , 1989 ) .",formal


In [42]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [43]:
learn = learn.load_encoder('finetuned')

In [44]:
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.451489,0.316294,0.978500,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.394637,0.117572,0.994500,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.301407,0.046659,0.993500,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.238748,0.026065,0.995000,00:01
1,0.230396,0.029757,0.991500,00:01


In [45]:
learn.predict("Blessed are the meek, for they shall inherit the Earth.")

('informal', TensorText(1), TensorText([0.0930, 0.9070]))

In [46]:
save_pickle("formality.pkl", learn)

In [50]:
learn.export("./learner.pkl")

In [48]:
!ls


academic.txt	   formality.pkl       LICENSE		 seinfeld.csv
chunk_seinfeld.py  formal.txt	       nohup.out	 seinfeld.txt
data		   gradio_front.ipynb  produce_texts.py  Transformers.ipynb
data_tok	   gradio_front.py     README.md	 Untitled.ipynb
day_1.ipynb	   informal.txt        resources.md
flagged		   languages.txt       RNN.ipynb
